In [1]:
import unittest
import pandas as pd



# Test Harness

In [2]:
import pandas_datareader as pdr
from datetime import datetime

start = datetime(year=2019, month=1, day=1)
end = datetime(year=2020, month=1, day=1)

yahoo_reader = pdr.yahoo.daily.YahooDailyReader(symbols=['AAPL', 'GOOG'], start=start, end=end, adjust_price=True, interval='d', get_actions=False, adjust_dividends=True)
test_data_df = yahoo_reader.read()
yahoo_reader.close()

In [3]:
test_data_df.head()

Attributes Adj_Ratio           Close                    High               \
Symbols         AAPL GOOG       AAPL         GOOG       AAPL         GOOG   
Date                                                                        
2019-01-02  0.969542  1.0  38.277519  1045.849976  38.502940  1052.319946   
2019-01-03  0.969542  1.0  34.464802  1016.059998  35.320422  1056.979980   
2019-01-04  0.969542  1.0  35.936081  1070.709961  36.006375  1070.839966   
2019-01-07  0.969542  1.0  35.856094  1068.390015  36.074244  1074.000000   
2019-01-08  0.969542  1.0  36.539616  1076.280029  36.798970  1084.560059   

Attributes        Low                    Open                    Volume  \
Symbols          AAPL         GOOG       AAPL         GOOG         AAPL   
Date                                                                      
2019-01-02  37.383116  1015.710022  37.543091  1016.570007  148158800.0   
2019-01-03  34.418748  1014.070007  34.898670  1041.000000  365248800.0   
2019-01-04  34.855043  1027.417969  35.031984  1032.589966  234428400.0   
2019-01-07  35.364052  1054.760010  36.042732  1071.500000  219111200.0   
2019-01-08  35.999097  1060.530029  36.251176  1076.109985  164101200.0   

Attributes             
Symbols          GOOG  
Date                   
2019-01-02  1532600.0  
2019-01-03  1841100.0  
2019-01-04  2093900.0  
2019-01-07  1981900.0  
2019-01-08  1764900.0

In [4]:
test_data_df.tail()

Attributes Adj_Ratio           Close                    High               \
Symbols         AAPL GOOG       AAPL         GOOG       AAPL         GOOG   
Date                                                                        
2019-12-24   0.98411  1.0  69.938202  1343.560059  70.090745  1350.260010   
2019-12-26   0.98411  1.0  71.325806  1360.400024  71.343029  1361.327026   
2019-12-27   0.98411  1.0  71.298737  1351.890015  72.324674  1364.530029   
2019-12-30   0.98411  1.0  71.721901  1336.140015  72.009756  1353.000000   
2019-12-31   0.98411  1.0  72.245941  1337.020020  72.253322  1338.000000   

Attributes        Low                    Open                    Volume  \
Symbols          AAPL         GOOG       AAPL         GOOG         AAPL   
Date                                                                      
2019-12-24  69.606071  1342.780029  70.041537  1348.500000   48478800.0   
2019-12-26  70.044005  1344.469971  70.073527  1346.170044   93121200.0   
2019-12-27  70.885412  1349.310059  71.623495  1362.989990  146266000.0   
2019-12-30  70.171931  1334.020020  71.215085  1350.000000  144114400.0   
2019-12-31  71.229847  1329.084961  71.330719  1330.109985  100805600.0   

Attributes             
Symbols          GOOG  
Date                   
2019-12-24   347500.0  
2019-12-26   667500.0  
2019-12-27  1038400.0  
2019-12-30  1050900.0  
2019-12-31   961800.0

In [5]:
test_data_df['Close'].head()

Symbols,AAPL,GOOG
Date,,
2019-01-02,38.277519,1045.849976
2019-01-03,34.464802,1016.059998
2019-01-04,35.936081,1070.709961
2019-01-07,35.856094,1068.390015
2019-01-08,36.539616,1076.280029


In [6]:
test_data_df['Close'].tail()

Symbols,AAPL,GOOG
Date,,
2019-12-24,69.938202,1343.560059
2019-12-26,71.325806,1360.400024
2019-12-27,71.298737,1351.890015
2019-12-30,71.721901,1336.140015
2019-12-31,72.245941,1337.020020


# Unit Tests

In [14]:
import importlib
import trading_factors_yahoo as alpha_factors
importlib.reload(alpha_factors)

class TestFactorData(unittest.TestCase):
    def test_init(self):
        class_under_test = alpha_factors.FactorData(test_data_df)
        self.assertEqual(class_under_test.factor_name, 'Alpha Factor')
        self.assertEqual(len(class_under_test.factor_data.columns), len(test_data_df.columns))
        
    def test_rank(self):
        class_under_test = alpha_factors.FactorReturns(test_data_df).rank()
        self.assertEqual(class_under_test.factor_data.loc['2019-01-03']['AAPL'], 1.0)
        self.assertEqual(class_under_test.factor_data.loc['2019-12-31']['AAPL'], 2.0)
    
    def test_demean(self):
        class_under_test = alpha_factors.FactorReturns(test_data_df).demean()
        self.assertAlmostEqual(class_under_test.factor_data.loc['2019-01-03']['AAPL'], -0.035561, places=4)
        self.assertAlmostEqual(class_under_test.factor_data.loc['2019-12-31']['AAPL'], 0.0033238, places=4)
        
    def test_zscore(self):
        class_under_test = alpha_factors.FactorReturns(test_data_df).zscore()
        self.assertAlmostEqual(class_under_test.factor_data.loc['2019-01-03']['AAPL'], -1.0, places=4)
        self.assertAlmostEqual(class_under_test.factor_data.loc['2019-12-31']['AAPL'], 1.0, places=4)
        
    def test_smoothed(self):
        class_under_test = alpha_factors.FactorReturns(test_data_df).smoothed(2)
        self.assertAlmostEqual(class_under_test.factor_data.loc['2019-01-04']['AAPL'], -0.028459, places=4)
        self.assertAlmostEqual(class_under_test.factor_data.loc['2019-12-31']['AAPL'], 0.006621, places=4)
        
    def test_for_al(self):
        series_data = alpha_factors.FactorReturns(test_data_df).for_al()
        self.assertEqual(series_data.name, 'returns_1_day')
        self.assertTupleEqual(series_data.shape, (502,))
        self.assertAlmostEqual(series_data.loc['2019-12-31', 'AAPL'], 0.007306, places=4)
        
    def test_open_values(self):
        class_under_test = alpha_factors.OpenPrices(test_data_df)
        self.assertEqual(class_under_test.factor_name, 'open')
        self.assertEqual(class_under_test.factor_data.columns[0], 'AAPL')
        self.assertAlmostEqual(class_under_test.factor_data.loc['2019-01-02']['AAPL'], 37.543095, places=4)
        self.assertAlmostEqual(class_under_test.factor_data.loc['2019-12-31']['AAPL'], 71.330711, places=4)

    def test_close_values(self):
        class_under_test = alpha_factors.ClosePrices(test_data_df)
        self.assertEqual(class_under_test.factor_name, 'close')
        self.assertEqual(class_under_test.factor_data.columns[0], 'AAPL')
        self.assertAlmostEqual(class_under_test.factor_data.loc['2019-01-02']['AAPL'], 38.277523, places=4)
        self.assertAlmostEqual(class_under_test.factor_data.loc['2019-12-31']['AAPL'], 72.245934, places=4)
        
    def test_volume_values(self):
        class_under_test = alpha_factors.Volume(test_data_df)
        self.assertEqual(class_under_test.factor_name, 'volume')
        self.assertEqual(class_under_test.factor_data.columns[0], 'AAPL')
        self.assertAlmostEqual(class_under_test.factor_data.loc['2019-01-02']['AAPL'], 148158800.0, places=4)
        self.assertAlmostEqual(class_under_test.factor_data.loc['2019-12-31']['AAPL'], 100805600.0, places=4)
        
    def test_returns(self):
        class_under_test = alpha_factors.FactorReturns(test_data_df)
        self.assertEqual(class_under_test.factor_name, 'returns_1_day')
        self.assertEqual(class_under_test.factor_data.columns[0], 'AAPL')
        self.assertTupleEqual(class_under_test.factor_data.shape, (252,2))
        self.assertAlmostEqual(class_under_test.factor_data.loc['2019-12-31']['AAPL'], 0.007306, places=4)
        
    def test_momentum(self):
        class_under_test = alpha_factors.FactorMomentum(test_data_df, 5)
        self.assertEqual(class_under_test.factor_name, 'momentum_5_day')
        self.assertEqual(class_under_test.factor_data.columns[0], 'AAPL')
        self.assertTupleEqual(class_under_test.factor_data.shape, (252,2))
        self.assertAlmostEqual(class_under_test.factor_data.loc['2019-12-31']['AAPL'], 0.033978, places=4)    
        
    def test_meanreversion(self):
        class_under_test = alpha_factors.FactorMeanReversion(test_data_df, 5)
        self.assertEqual(class_under_test.factor_name, 'mean_reversion_5_day_logret')
        self.assertEqual(class_under_test.factor_data.columns[0], 'AAPL')
        self.assertTupleEqual(class_under_test.factor_data.shape, (252,2))
        self.assertAlmostEqual(class_under_test.factor_data.loc['2019-12-31']['AAPL'], -0.033978, places=4)

In [15]:
TestFactorData().test_volume_values()

In [16]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_close_values (__main__.TestFactorData) ... ok
test_demean (__main__.TestFactorData) ... ok
test_for_al (__main__.TestFactorData) ... ok
test_init (__main__.TestFactorData) ... ok
test_meanreversion (__main__.TestFactorData) ... ok
test_momentum (__main__.TestFactorData) ... ok
test_open_values (__main__.TestFactorData) ... ok
test_rank (__main__.TestFactorData) ... ok
test_returns (__main__.TestFactorData) ... ok
test_smoothed (__main__.TestFactorData) ... ok
test_volume_values (__main__.TestFactorData) ... ok
test_zscore (__main__.TestFactorData) ... ok

----------------------------------------------------------------------
Ran 12 tests in 0.087s

OK
